In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from matplotlib.pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import stats
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

import jpx_tokyo_market_prediction

In [ ]:
#financials = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv")
#stock_list = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
#financials = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv")
#options = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/options.csv")
#sprices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/secondary_stock_prices.csv")
supplemental_prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
#supplemental_sprices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/secondary_stock_prices.csv")

In [ ]:
prices=prices.append(supplemental_prices,ignore_index=True)

In [ ]:
prices['DateValue']=prices['Date'].str.replace('-','')
small_prices = prices[prices['DateValue']>'20180620']
small_prices=small_prices.drop(['DateValue'],axis=1)
small_prices.info()

# Feature Engineering Functions

In [ ]:
def feature_train(train):
    
    #train = train.groupby("SecuritiesCode")
    
    # Add Lag Features
    lag_features = ["High", "Low", "Volume", "Close", "Open"]
    df_rolled_7d = train[lag_features].rolling(window=4, min_periods=0)
    df_mean_7d = df_rolled_7d.mean().shift(1).reset_index().astype(np.float32)
    df_mean_7d = df_mean_7d.drop('index', axis=1)
    df_mean_7d = df_mean_7d.fillna(0)
    df_mean_7d = df_mean_7d.round(2)
    df_mean_7d['High_lag_1'] = df_mean_7d['High']
    df_mean_7d['Open_lag_1'] = df_mean_7d['Open']
    df_mean_7d['Close_lag_1'] = df_mean_7d['Close']
    df_mean_7d['Volume_lag_1'] = df_mean_7d['Volume']
    df_mean_7d['Low_lag_1'] = df_mean_7d['Low']
    df_mean_7d = df_mean_7d.drop(["High", "Low", "Volume", "Close", "Open"], axis=1)
    train = train.reset_index(drop=True)
    train = pd.concat([train, df_mean_7d], axis=1)

    # Convert Date to float
    train['Date_Float'] = train['Date'].str.replace('-', '')
    train['Date_Float'] = train['Date_Float'].astype(float) 
    
    # Drop irrelevant columns for training
    train = train.drop(
                        ['RowId', 
                         'SupervisionFlag',
                         'AdjustmentFactor'], axis=1)
    
    # Bool to int for SupervisionFlag
    #train["SupervisionFlag"] = train["SupervisionFlag"].astype(int)
    
    # Backward, then forward fill missing values in cols
    cols = ['Target','Open', 'High', 'Low', 'Close']
    train.loc[:,cols] = train.loc[:,cols].bfill()
    train.loc[:,cols] = train.loc[:,cols].ffill()
    
    # Add Spread Features
    train['Daily_Spread'] = train['Close'] - train['Open']
    train['Daily_Max_Min'] = train['High'] - train['Low']
    train['1_Day_Spread_Close'] = train['Close'].diff()
    train['2_Day_Spread_Close'] = train['Close'].diff(periods=2)
    train['1_Day_Spread_Open'] = train['Open'].diff()
    train['2_Day_Spread_Open'] = train['Open'].diff(periods=2)
    train['1_Week_Spread'] = train['Close'].diff(periods=5)
    
    # Fill NaN's 
    train = train.fillna(0)
    
    # Add Return Features
    train['Return_Lag_1_Close'] = (train['Close'] - train['1_Day_Spread_Close'])/train['Close']
    train['Return_Lag_2_Close'] = (train['Close'] - train['2_Day_Spread_Close'])/train['Close']
    
    train['Return_Lag_1_Open'] = (train['Open'] - train['1_Day_Spread_Open'])/train['Open']
    train['Return_Lag_2_Open'] = (train['Open'] - train['2_Day_Spread_Open'])/train['Open'] 

    # Fill missing and inf/-inf values with 0
    train.replace([np.inf, -np.inf], 0, inplace=True)
    train = train.fillna(0)
        
    # Add rolling ratio of mean/std of forward 1 day return
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=5)
    train['ExPost_SR_Close'] = (train['Return_Lag_1_Close'].rolling(
        window=indexer, min_periods=1).mean())/(
        train['Return_Lag_1_Close'].std())
    
    train['ExPost_SR_2_Close'] = (train['Return_Lag_2_Close'].rolling(
        window=indexer, min_periods=1).mean())/(
        train['Return_Lag_2_Close'].std())
    
    train['ExPost_SR_Open'] = (train['Return_Lag_1_Open'].rolling(
        window=indexer, min_periods=1).mean())/(
        train['Return_Lag_1_Open'].std())
    
    train['ExPost_SR_2_Open'] = (train['Return_Lag_2_Open'].rolling(
        window=indexer, min_periods=1).mean())/(
        train['Return_Lag_2_Open'].std())
    
    # Fill missing and inf/-inf values with 0
    train.replace([np.inf, -np.inf], 0, inplace=True)
    train = train.fillna(0)
    
    # Fill missing values with 0
    #train = train.fillna(0)
    
    return train

# Min Max Scaler Function

In [ ]:
def min_max(df):
    # MinMax Scale columns (-1, 1 scale)   
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled = scaler.fit_transform(df)
    train_cols = df.columns.values.tolist()
    trained = pd.DataFrame(data=scaled, columns=train_cols, index=df.index)
    return trained

# Build Training DataFrame Function

In [ ]:
def build_train(df):
    # Groupby for feature engineering by securities code
    stock_list_df = df.groupby("SecuritiesCode").apply(feature_train)

    # reset index
    stock_list_df = stock_list_df.reset_index(drop=True)

    # Create Securities Code df
    sec_code_df = stock_list_df[['SecuritiesCode', 'Date']]
    
    # Drop Date column
    stock_list_df = stock_list_df.drop('Date', axis=1)

    # min max scale prices df
    stock_list_df = min_max(stock_list_df)

    # drop SecuritiesCode column before adding it back
    stock_list_df = stock_list_df.drop(['SecuritiesCode'], axis=1)

    # Concat dfs together
    stock_list_df = pd.concat([sec_code_df, stock_list_df], axis=1)

    # Sort df by date
    stock_list_df = stock_list_df.sort_values(by='Date')

    return stock_list_df

# Submission

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [ ]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    date_df = prices[['Date', 'SecuritiesCode']]

    # Append small_prices and prices
    concat_df = small_prices.append(prices)
    train_df = build_train(concat_df) 

    # Create test_df from prices - date_df
    df_test = pd.merge(date_df, train_df, how='left', on=["Date","SecuritiesCode"])
    df_test = df_test.drop('Target', axis=1)

    # Drop Date column from df_test
    df_test = df_test.drop('Date', axis=1)

    # Cut out last date as it is likely the prices data
    single_model = train_df.loc[train_df['Date_Float'] != 1.0]

    # Drop Date column from multi_model
    single_model = single_model.drop('Date', axis=1)
    
    # create Train X and y df's
    X_train = single_model.drop('Target', axis=1)
    y_train = single_model['Target']

    # create Test X and y df's
    X_test = df_test

    # Instantiate and Fit Model
    rf = RandomForestRegressor(criterion="absolute_error", random_state = 42)

    # Train the model on training data
    rf.fit(X_train, y_train);

    # df of preds by sec code
    predictions = rf.predict(X_test)

    # Create predictions df
    sample_prediction['Target'] = predictions
    
    # Sort predictions df by Target
    sample_prediction = sample_prediction.sort_values(by = "Target", ascending = False)
    
    # Rank predictions df by Target, create Rank column
    sample_prediction['Rank'] = np.arange(len(sample_prediction.index))
    
    # Sort predictions column by Securities Code
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending = True)
    
    # Drop Target feature
    sample_prediction.drop(["Target"], axis = 1)
    
    # Create Submission df
    submission = sample_prediction[["Date", "SecuritiesCode", "Rank"]]    
    
    env.predict(submission)

In [ ]:
print(prices)

In [ ]:
submission